In [1]:
# Copyright 2021 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# =====

## 3. Building Multi-Stage Recommender Systems

Recommender Systems (RecSys) are the engine of the modern internet and the catalyst for human decisions. Building a recommendation system is challenging because it requires multiple stages (data preprocessing, offline training, item retrieval, filtering, ranking, ordering, etc.) to work together seamlessly and efficiently. The biggest challenges for new practitioners are the lack of understanding around what RecSys look like in the real world, and the gap between examples of simple models and a production-ready end-to-end recommender systems.

The figure below represents a four-stage recommender systems. This is more complex process than only training a single model and deploying it, and it is much more realistic and closer to what's happening in the real-world recommender production systems.

In this lab, we are going to showcase how we can deploy a four-stage recommender systems using Merlin Systems library easily on Triton Inference Server. Let's go over the concepts in the figure briefly.

- **Retrieval:** This is the step to narrow down millions of items into thousands of candidates. We are going to train a Two-Tower item retrieval model to retrieve the relevant top-K candidate items.
- **Filtering:** This step is to exclude the already interacted or undesirable items from the candidate items set or to apply business logic rules. Although this is an important step, for this example we skip this step.
- **Scoring:** This is also known as ranking. Here the retrieved and filtered candidate items are being scored. We are going to train a ranking model to be able to use at our scoring step.
- **Ordering:** At this stage, we can order the final set of items that we want to recommend to the user. Here, we’re able to align the output of the model with business needs, constraints, or criteria.

To learn more about the four-stage recommender systems, you can listen to Even Oldridge's [Moving Beyond Recommender Models talk at KDD'21](https://www.youtube.com/watch?v=5qjiY-kLwFY&list=PL65MqKWg6XcrdN4TJV0K1PdLhF_Uq-b43&index=8) and read more in this [blog post](https://eugeneyan.com/writing/system-design-for-discovery/).

**Learning Objectives of this notebook**

- Train a ranking and retriveal model with Merlin Models
- Export user query tower, user and item features, and item embedding
- Create feature store with Feast and register features

**Import Required Libraries**

In [1]:
import os

import glob
import cudf 
import pandas as pd
import numpy as np
import nvtabular as nvt
from nvtabular.ops import *
import gc
from datetime import datetime

from merlin.schema.tags import Tags
import merlin.models.tf as mm
from merlin.io.dataset import Dataset
from merlin.models.utils.dataset import unique_rows_by_features

import tensorflow as tf

2022-08-18 23:41:37.244178: I tensorflow/core/platform/cpu_feature_guard.cc:194] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-18 23:41:39.903778: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 8080 MB memory:  -> device: 0, name: Tesla V100-SXM2-16GB-N, pci bus id: 0000:06:00.0, compute capability: 7.0
2022-08-18 23:41:39.920047: I tensorflow/stream_executor/cuda/cuda_driver.cc:739] failed to allocate 7.89G (8472494080 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2022-08-18 23:41:39.922848: I tensorflow/stream_executor/cuda/cuda_driver.cc:739] failed to allocate 7.10G (7625244672 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2022-08-18 23:41:39.92586

In [2]:
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

In [3]:
data_path = '/workspace/data/ecom/'
output_path = os.path.join(data_path,'processed_nvt')
output_path2 = os.path.join(data_path,'processed_filtered')
BASE_DIR = os.environ.get(
    "BASE_DIR", "/workspace/recsys_tutorial/"
)

Read processed parquet files as Dataset objects.

In [5]:
train_raw = Dataset(os.path.join(output_path, "train", "*.parquet"), part_size="500MB")
valid_raw = Dataset(os.path.join(output_path, "valid", "*.parquet"), part_size="500MB")

/usr/local/lib/python3.8/dist-packages/cudf/core/frame.py:384: UserWarning: The deep parameter is ignored and is only included for pandas compatibility.
  warnings.warn(


**Filter out the negative rows**

Here, we will filter our datasets with NVTabular `Filter()` operator to select only positive interaction rows where `target==1` in the dataset. We do that because we want to use `negative sampling` technique when training our candidate retrieval and ranking models.

In [6]:
inputs = train_raw.schema.column_names
outputs = inputs >> Filter(f=lambda df: df["target"] == 1)

In [7]:
workflow2 = nvt.Workflow(outputs)

workflow2.fit(train_raw)

workflow2.transform(train_raw).to_parquet(
    output_path=os.path.join(output_path2, "train")
)

workflow2.transform(valid_raw).to_parquet(
    output_path=os.path.join(output_path2, "valid")
)

In [8]:
workflow2.save(os.path.join(output_path2, "workflow2"))

**Read filtered parquet files as Dataset objects.**

In [4]:
train = Dataset(os.path.join(output_path2, "train", "*.parquet"), part_size="500MB")
valid = Dataset(os.path.join(output_path2, "valid", "*.parquet"), part_size="500MB")

/usr/local/lib/python3.8/dist-packages/cudf/core/frame.py:384: UserWarning: The deep parameter is ignored and is only included for pandas compatibility.
  warnings.warn(


In [5]:
train.schema

,name,tags,dtype,is_list,is_ragged,properties.num_buckets,properties.freq_threshold,properties.max_size,properties.start_index,properties.cat_path,properties.embedding_sizes.cardinality,properties.embedding_sizes.dimension,properties.domain.min,properties.domain.max
0,user_id,"(Tags.USER_ID, Tags.CATEGORICAL, Tags.USER)",int32,False,False,NaN,0.0,0.0,0.0,.//categories/unique.user_id.parquet,351050.0,512.0,0.0,351050.0
1,ts_weekday,"(Tags.CATEGORICAL, Tags.USER)",int32,False,False,NaN,0.0,0.0,0.0,.//categories/unique.ts_weekday.parquet,8.0,16.0,0.0,8.0
2,ts_hour,"(Tags.CATEGORICAL, Tags.USER)",int32,False,False,NaN,0.0,0.0,0.0,.//categories/unique.ts_hour.parquet,25.0,16.0,0.0,25.0
3,product_id,"(Tags.CATEGORICAL, Tags.ITEM, Tags.ITEM_ID)",int32,False,False,NaN,0.0,0.0,0.0,.//categories/unique.product_id.parquet,51425.0,512.0,0.0,51425.0
4,cat_0,"(Tags.ITEM, Tags.CATEGORICAL)",int32,False,False,NaN,0.0,0.0,0.0,.//categories/unique.cat_0.parquet,14.0,16.0,0.0,14.0
5,cat_1,"(Tags.ITEM, Tags.CATEGORICAL)",int32,False,False,NaN,0.0,0.0,0.0,.//categories/unique.cat_1.parquet,61.0,16.0,0.0,61.0
6,cat_2,"(Tags.ITEM, Tags.CATEGORICAL)",int32,False,False,NaN,0.0,0.0,0.0,.//categories/unique.cat_2.parquet,90.0,20.0,0.0,90.0
7,brand,"(Tags.ITEM, Tags.CATEGORICAL)",int32,False,False,NaN,0.0,0.0,0.0,.//categories/unique.brand.parquet,2654.0,132.0,0.0,2654.0
8,price,"(Tags.ITEM, Tags.CONTINUOUS)",float32,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,relative_price,"(Tags.ITEM, Tags.CONTINUOUS)",float32,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Building and Training a Candidate Retrieval Model with Merlin Models

Industrial recommender systems have major tasks to accomplish that can be quite demanding. One requirement is to deliver a recommendation under the expected latency requirements (e.g., within milliseconds) to warrant a good user experience. That might require a significant amount of creativity and engineering. And the second consideration is that we might want to minimize infrastructure costs while solving the latency issue, which is yet another obstacle to overcome!

In large scale recommender systems pipelines, the size of the item catalog (number of unique items) might be in the order of millions or billions. At such scale, a typical setup is having two-stage pipeline, where a faster candidate retrieval model quickly extracts thousands of relevant items and a then a more powerful ranking model (i.e. with more features and more powerful architecture) ranks the top-k items that are going to be displayed to the user. 

Therefore, industrial recommender systems usually consists of candidate retrieval and ranking (scoring) stages. The candidate retrieval stage retrieves candidate items that are relevant to user interests, while the ranking stage sorts candidate items by user interests.

In this notebook, we start with the first stage of multi-stage recommender systems- the Candidate Retrieval. For ML-based candidate retrieval model, as it needs to quickly score millions of items for a given user, a popular choices are models that can produce recommendation scores by just computing the dot product the user embeddings and item embeddings. Popular choices of such models are `Matrix Factorization (MF)`, which learns low-rank user and item embeddings, and the `Two-Tower architecture`, which is a neural network with two MLP towers where both user and item features are fed to generate user and item embeddings in the output. Such models can be efficiently served by indexing the trained item embeddings into an Approximate Nearest Neighbors (ANN) engine and during inference scoring user embeddings over all indexed item embeddings within the engine. 

#### Two-Tower Model

We are going to train a Two-Tower model for item retrieval. A Two-Tower Model consists of item (candidate) and user (query) encoder towers. With two towers, the model can learn representations (embeddings) for queries and candidates separately.

--- ADD IMAGE---

Now, let's build our Two-Tower model. In a nutshell, we aggregate all user features to feed in user tower and feed the item features to the item tower. Then we compute the positive score by multiplying the user embedding with the item embedding and sample negative items (read more about negative sampling here and here), whose item embeddings are also multiplied by the user embedding. Then we apply the loss function on top of the positive and negative scores.

In [10]:
schema = train.schema
schema = schema.select_by_tag([Tags.ITEM_ID, Tags.USER_ID, Tags.ITEM, Tags.USER])

In [11]:
model_tt = mm.TwoTowerModel(
    schema,
    query_tower=mm.MLPBlock([128, 64], no_activation_last_layer=True),
    samplers=[mm.InBatchSampler()],
    embedding_options=mm.EmbeddingOptions(infer_embedding_sizes=True),
)

In [12]:
%%time
model_tt.compile(
    optimizer="adam",
    run_eagerly=False,
    loss="categorical_crossentropy",
    metrics=[mm.RecallAt(10), mm.NDCGAt(10)],
)
model_tt.fit(train, validation_data=valid, batch_size=1024 * 8, epochs=2)

Epoch 1/2
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The sampler InBatchSampler returned no samples for this batch.


190/191 [============================>.] - ETA: 0s - loss: 8.4344 - recall_at_10: 0.0168 - ndcg_at_10: 0.0149 - regularization_loss: 0.0000e+00

The sampler InBatchSampler returned no samples for this batch.


191/191 [==============================] - 30s 58ms/step - loss: 8.4353 - recall_at_10: 0.0169 - ndcg_at_10: 0.0150 - regularization_loss: 0.0000e+00 - val_loss: 8.3632 - val_recall_at_10: 0.0108 - val_ndcg_at_10: 0.0062 - val_regularization_loss: 0.0000e+00
Epoch 2/2
191/191 [==============================] - 10s 49ms/step - loss: 7.6621 - recall_at_10: 0.0368 - ndcg_at_10: 0.0279 - regularization_loss: 0.0000e+00 - val_loss: 8.2335 - val_recall_at_10: 0.0165 - val_ndcg_at_10: 0.0087 - val_regularization_loss: 0.0000e+00
CPU times: user 1min 47s, sys: 4.9 s, total: 1min 52s
Wall time: 42.3 s


#### Exporting query (user) model

We export the query tower to use it later during the model deployment stage with Merlin Systems.

In [13]:
query_tower = model_tt.retrieval_block.query_block()
query_tower.save(os.path.join(BASE_DIR, "query_tower"))

INFO:tensorflow:Assets written to: /workspace/recsys_tutorial/query_tower/assets


INFO:tensorflow:Assets written to: /workspace/recsys_tutorial/query_tower/assets


### Train a ranking Model

In [14]:
schema = train.schema.without(['event_time_ts'])
target_column = schema.select_by_tag(Tags.TARGET).column_names[0]
target_column

'target'

In [15]:
# Augment the batch of positive interactions with `n_per_positive` negatives sampled from the same batch. 
from merlin.models.tf.data_augmentation.negative_sampling import UniformNegativeSampling
from merlin.models.tf.dataset import BatchedDataset

batch_size, n_per_positive = 2048, 8
add_negatives = UniformNegativeSampling(schema, n_per_positive, seed=42, return_tuple=True)
dataset = BatchedDataset(train, batch_size=batch_size, shuffle=True)
dataset = dataset.map(add_negatives)

In [16]:
train.to_ddf().compute().target.value_counts()

1    1557666
Name: target, dtype: int32

In [17]:
model = mm.DLRMModel(
    schema,
    embedding_dim=32,
    bottom_block=mm.MLPBlock([64, 32]),
    top_block=mm.MLPBlock([64, 32]),
    prediction_tasks=mm.BinaryClassificationTask(target_column),
)

In [18]:
%%time
model.compile(optimizer='adam', run_eagerly=False, metrics=[], 
              weighted_metrics=[tf.keras.metrics.BinaryAccuracy(),tf.keras.metrics.AUC()]
             )
model.fit(dataset, epochs=1, train_metrics_steps=100, run_eagerly=False)
#model.fit(dataset, epochs=2, class_weight = {0: 1, 1: n_per_positive}, train_metrics_steps=100)

761/761 [==============================] - 56s 66ms/step - loss: 0.3232 - binary_accuracy: 0.8046 - auc: 0.6624 - regularization_loss: 0.0000e+00
CPU times: user 1min 13s, sys: 5.18 s, total: 1min 18s
Wall time: 56.6 s


In [19]:
valid_dataset = BatchedDataset(valid, shuffle=False, batch_size = batch_size)
valid_dataset = valid_dataset.map(add_negatives)
metrics = model.evaluate(valid_dataset, return_dict=True)
metrics

47/47 [==============================] - 4s 64ms/step - loss: 0.3155 - binary_accuracy: 0.8825 - auc: 0.7689 - regularization_loss: 0.0000e+00


{'loss': 0.31547340750694275,
 'binary_accuracy': 0.8824513554573059,
 'auc': 0.7688804864883423,
 'regularization_loss': 0.0}

In [20]:
model.save(os.path.join(BASE_DIR, "dlrm"))

INFO:tensorflow:Assets written to: /workspace/recsys_tutorial/dlrm/assets


INFO:tensorflow:Assets written to: /workspace/recsys_tutorial/dlrm/assets


In [6]:
reloaded = tf.keras.models.load_model(os.path.join(BASE_DIR, "dlrm"))

In [7]:
default_signature = reloaded.signatures["serving_default"]

In [8]:
default_signature

<ConcreteFunction signature_wrapper(*, TE_brand_target, TE_cat_1_target, TE_cat_2_target, TE_user_id_target, brand, cat_0, cat_1, cat_2, price, product_id, relative_price, ts_hour, ts_weekday, user_id) at 0x7F25D8725040>

#### Set up a feature store with Feast

In [21]:
!rm -rf $BASE_DIR/feature_repo
!cd $BASE_DIR && feast init feature_repo


Creating a new Feast repository in /workspace/recsys_tutorial/feature_repo.



Feast is an open source project that collects anonymized error reporting and usage statistics. To opt out or learn more see https://docs.feast.dev/reference/usage

Creating a new Feast repository in /Merlin/examples/Building-and-deploying-multi-stage-RecSys/feature_repo.

You should be seeing a message like Creating a new Feast repository in ... printed out above. Now, navigate to the feature_repo folder and remove the demo parquet file created by default, and examples.py file.

In [22]:
feature_repo_path = os.path.join(BASE_DIR, "feature_repo")
if os.path.exists(f"{feature_repo_path}/example.py"):
    os.remove(f"{feature_repo_path}/example.py")
if os.path.exists(f"{feature_repo_path}/data/driver_stats.parquet"):
    os.remove(f"{feature_repo_path}/data/driver_stats.parquet")

In [23]:
user_features = (
    unique_rows_by_features(train_raw, [Tags.USER,Tags.TIME], Tags.USER_ID)
    .compute()
    .reset_index(drop=True)
)

In [25]:
user_features.user_id.nunique(), user_features.user_id.max()

(351049, 351049)

In [ ]:
user_features["datetime"] = user_features["event_time_ts"].astype("datetime64[ns]")
user_features["created"] = datetime.now()
user_features["created"] = user_features["created"].astype("datetime64[ns]")

In [ ]:
user_features = user_features.drop(columns=['event_time_ts'])

In [31]:
user_features.head()

,user_id,ts_weekday,ts_hour,datetime,created
0,1,2,6,2020-03-15 11:47:05,2022-08-18 23:22:26.096830
1,2,2,9,2020-03-29 13:19:05,2022-08-18 23:22:26.096830
2,3,3,1,2020-03-21 08:28:23,2022-08-18 23:22:26.096830
3,4,2,6,2020-03-29 11:14:14,2022-08-18 23:22:26.096830
4,5,6,8,2019-12-26 12:14:27,2022-08-18 23:22:26.096830


In [32]:
user_features.to_parquet(
    os.path.join(BASE_DIR, "feature_repo/data", "user_features.parquet")
)

In [33]:
item_features = (
    unique_rows_by_features(train_raw, [Tags.ITEM, Tags.TIME], Tags.ITEM_ID)
    .compute()
    .reset_index(drop=True)
)

In [34]:
item_features["datetime"] = item_features["event_time_ts"].astype("datetime64[ns]")
item_features["created"] = datetime.now()
item_features["created"] = item_features["created"].astype("datetime64[ns]")

In [35]:
item_features = item_features.drop(columns=['event_time_ts'])

In [36]:
item_features.head(2)

,product_id,cat_0,cat_1,cat_2,brand,price,relative_price,TE_user_id_target,TE_brand_target,TE_cat_1_target,TE_cat_2_target,datetime,created
0,1,1,1,1,1,0.470813,0.057298,0.377244,0.520552,0.572554,0.909697,2020-03-31 16:24:49,2022-08-18 23:23:11.983316
1,2,1,1,1,2,1.543032,0.084794,-0.895478,0.451823,0.571428,0.908887,2020-03-31 14:33:00,2022-08-18 23:23:11.983316


In [37]:
# save to disk
item_features.to_parquet(
    os.path.join(BASE_DIR, "feature_repo/data", "item_features.parquet")
)

In [38]:
item_embs = model_tt.item_embeddings(
    Dataset(item_features, schema=schema), batch_size=1024
)
item_embs_df = item_embs.compute(scheduler="synchronous")

INFO:tensorflow:Assets written to: /tmp/tmpea4gdz0r/assets


INFO:tensorflow:Assets written to: /tmp/tmpea4gdz0r/assets


In [39]:
# select only item_id together with embedding columns
item_embeddings = item_embs_df.drop(
    columns=['cat_0', 'cat_1', 'cat_2', 'brand', 'price',
       'relative_price', 'TE_user_id_target', 'TE_brand_target',
       'TE_cat_1_target', 'TE_cat_2_target']
)

In [40]:
item_embeddings.head(2)

,product_id,0,1,2,3,4,5,6,7,8,...,54,55,56,57,58,59,60,61,62,63
0,1,-0.147541,-0.420399,-0.180685,0.772281,0.376931,-0.02216,-0.074293,-0.689154,0.428960,...,-0.977655,-0.262428,-0.855970,0.273959,-0.703504,-0.625795,-0.210638,0.019891,-2.096206,-1.192018
1,2,-0.215549,-0.417371,-0.870647,0.428005,-0.395210,-0.52977,0.713359,0.318280,1.491366,...,0.628782,0.439529,-0.311931,-1.121799,-0.008436,-0.649124,0.583354,-0.175015,-2.881193,-1.822508


In [41]:
# save to disk
item_embeddings.to_parquet(os.path.join(BASE_DIR, "item_embeddings.parquet"))

### Create feature definitions

Now we will create our user and item features definitions in the user_features.py and item_features.py files and save these files in the feature_repo.

In [44]:
file = open(os.path.join(BASE_DIR, "feature_repo/", "user_features.py"), "w")
file.write(
    """
from google.protobuf.duration_pb2 import Duration
import datetime
from feast import Entity, Feature, FeatureView, ValueType
from feast.infra.offline_stores.file_source import FileSource

user_features = FileSource(
    path="{}",
    event_timestamp_column="datetime",
    created_timestamp_column="created",
)

user = Entity(name="user_id", value_type=ValueType.INT32, description="user id",)

user_features_view = FeatureView(
    name="user_features",
    entities=["user_id"],
    ttl=Duration(seconds=86400 * 7),
    features=[
        Feature(name="ts_weekday", dtype=ValueType.INT32),
        Feature(name="ts_hour", dtype=ValueType.INT32),
    ],
    online=True,
    input=user_features,
    tags=dict(),
)
""".format(
        os.path.join(BASE_DIR, "feature_repo/data/", "user_features.parquet")
    )
)
file.close()

In [45]:
with open(os.path.join(BASE_DIR, "feature_repo/", "item_features.py"), "w") as f:
    f.write(
        """
from google.protobuf.duration_pb2 import Duration
import datetime
from feast import Entity, Feature, FeatureView, ValueType
from feast.infra.offline_stores.file_source import FileSource

item_features = FileSource(
    path="{}",
    event_timestamp_column="datetime",
    created_timestamp_column="created",
)

item = Entity(name="product_id", value_type=ValueType.INT32, description="product id",)

item_features_view = FeatureView(
    name="item_features",
    entities=["product_id"],
    ttl=Duration(seconds=86400 * 7),
    features=[
        Feature(name="cat_0", dtype=ValueType.INT32),
        Feature(name="cat_1", dtype=ValueType.INT32),
        Feature(name="cat_2", dtype=ValueType.INT32),
        Feature(name="brand", dtype=ValueType.INT32),
        Feature(name="price", dtype=ValueType.FLOAT),
        Feature(name="relative_price", dtype=ValueType.FLOAT),
        Feature(name="TE_user_id_target", dtype=ValueType.FLOAT),
        Feature(name="TE_brand_target", dtype=ValueType.FLOAT),
        Feature(name="TE_cat_1_target", dtype=ValueType.FLOAT),
        Feature(name="TE_cat_2_target", dtype=ValueType.FLOAT),
    ],
    online=True,
    input=item_features,
    tags=dict(),
)
""".format(
            os.path.join(BASE_DIR, "feature_repo/data/", "item_features.parquet")
        )
    )
file.close()

### Summary 

In this hands-on lab we learned ...